# Acoustic 3D

This notebook demonstrates a 3D acoustic wave simulation for scattering from a spherical fluid scatterer. We'll guide you through the setup process step by step.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

from ultrawave import TimeAxis, ToneBurstSource, Receiver
from ultrawave.lib.model_3d import Model
from ultrawave.lib.operator import Acoustic3DOperator

## GPU acceleration

To use GPU to accelerate this simulation, you need to have NVIDIA HPC SDK installed, and run the following code.

If you don't want to use GPU, just skip this part. The code will automatically ran on CPU.


In [ ]:
from devito import configuration
configuration['platform'] = 'nvidiaX'
configuration['compiler'] = 'pgcc'
configuration['language'] = 'openacc'

## Define the model

The model encapsulates the simulation grid and medium properties:

- **Grid Setup**: We start by defining a grid with specified spacing and shape.
- **Background Medium**: A homogeneous medium with a sound speed of $1500 \text{ m/s}$ and a density of $1000 \text{ kg/m}^3$.


In [ ]:
# Define grid spacing in [m]
spacing = (5.e-5, 5.e-5, 5.e-5) # [m]

# The number of grid points in each dimension
shape = (int(20e-3/spacing[0]),
         int(20e-3/spacing[1]),
         int(8e-3/spacing[2]))

# Define the origin coordinate, which is the top left corner of the grid
origin = (0., 0., 0.)

# Compressional wave speed
vp_background = 1500  # Background speed in [m/s].
vp = np.full(shape, vp_background, dtype=np.float32)

# Density
rho_background = 1000  # Background density in [kg/m^3].
rho = np.full(shape, rho_background, dtype=np.float32)

- **Spherical Fluid Scatterer**:
  - **Location**: Centered in the middle of the grid.
  - **Radius**: $0.5 \text{ mm}$.
  - **Properties**: A sound speed of $1520 \text{ m/s}$ and a density of $1050 \text{ kg/m}^3$.

In [ ]:
# Define a spherical scatterer.
a = 0.5e-3 # Radius of the sphere in [m].
r = int(a / spacing[0])  # Radius of the sphere in grid points.
center_x, center_y, center_z = (shape[0] // 2, shape[1] // 2, shape[2] // 2) # Center of the sphere.
x, y, z = np.ogrid[-center_x:shape[0]-center_x,
                   -center_y:shape[1]-center_y,
                   -center_z:shape[2]-center_z]

mask = (x**2 + y**2 + z**2 <= r**2)
vp_scatter = 1520     # Speed of scatterer in [m/s].
rho_scatter = 1050     # Density of scatterer in [kg/m^3].
vp[mask] = vp_scatter
rho[mask] = rho_scatter

With the simulating grid and medium defined, we can create a 3D model with a time order of 2 and space order of 4. The size of boudary layers, also the perfectly matched layers, is 20 grid points.

In [ ]:
time_order = 2
space_order = 4
nbl = 20  # Number of boundary layers. Size of PML

# Define simulation time.
t0 = 0.0  # Start time of the simulation.
tn = 1.e-5  # End time of the simulation [s].
dt = 2e-9  # Time step [s].
time_range = TimeAxis(start=t0, stop=tn, step=dt)
nt = time_range.num

# Create a model
model = Model(vp=vp, rho=rho, origin=origin, shape=shape, spacing=spacing, space_order=space_order, dt=dt, nbl=nbl)


## Define the source

A planar source transmits a 3-cycle tone burst plane wave at a central frequency of $2 \text{ MHz}$. The planar source covers a range from $0$ to $20 \text{ mm}$ in both x and y directions, with a fixed z-coordinate at $1 \text{ mm}$.

The source coordinates are defined out of grid and in the unit of meter. Here, for convenience, we make spacing between neighboring source points uniform and matching the grid spacing. The total number of source points is calculated by $(20 \text{ mm}/\text{spacing})^2$.


In [ ]:
# Define a planar source.
f0 = 2e6 # Central frequency in [Hz]
src_npoints = int(20.e-3 / spacing[0]) * int(20.e-3 / spacing[1])
src = ToneBurstSource(name='src', grid=model.grid, f0=f0, cycles=3, npoint=src_npoints, time_range=time_range) # source signal is a 3-cycle tone burst

# Define source coordinates
x = np.arange(0., 20.e-3, spacing[0])
y = np.arange(0., 20.e-3, spacing[0])
xv, yv = np.meshgrid(x, y)
src.coordinates.data[:, 0] = np.reshape(xv, -1) # Position in the x-direction. [m]
src.coordinates.data[:, 1] = np.reshape(yv, -1) # Position in the y-direction. [m]
src.coordinates.data[:, 2] = 1.e-3  # Position in the z-direction. [m]
r_src = 4.e-3 - 1.e-3 # distance from source to scatterer

## Define the receiver
The definition of receiver is similar to source. We define a point receiver above the scatterer to receive the backscattered signal.

In [ ]:
# Define a point receiver
rec = Receiver(name='rec', grid=model.grid, npoint=1, time_range=time_range)
rec.coordinates.data[:, 0] = 20.e-3/2
rec.coordinates.data[:, 1] = 20.e-3/2
rec.coordinates.data[:, 2] = 1.e-3
r_rec = 4.e-3 - 1.e-3 # distance from receiver to scatterer

## Operator
Operator acts as the computational engine of UltraWave. It takes the model, source, and receiver configurations as input and implement the wave equations.

In [ ]:
# Define the operator
op = Acoustic3DOperator(model, source=src, reciever=rec)

# Run the operator
op(time=time_range.num-1, dt=dt)
rec_signal = rec.data

In [ ]:
rec_signal = np.load('acoustic_3d_rec_signal.npy')

Display the received signal

In [ ]:
time = np.linspace(t0, tn, nt)
plt.figure(figsize=(12,8))
plt.plot(time, rec_signal, '-r')
plt.title('Received signal')
plt.xlabel('time (s)')
plt.show()

We substract the incident wave from the received signal to obtain the scattered signal.

To get the incident wave, we need to run a reference simulation without the scatterer. Here, we just load the presaved data.

In [ ]:
incident_signal = np.load('acoustic_3d_incident_signal.npy') # load the presaved incident wave

normalized_scatter_signal = (rec_signal - incident_signal)/np.max(incident_signal) # normalized scattered signal

plt.figure(figsize=(12,8))
plt.plot(time, normalized_scatter_signal, '-r')
plt.title('Normalized scattered signal')
plt.xlabel('time (s)')
plt.xlim((2e-6, 8e-6))
plt.show()

## Theoretical validation

We validate the simulation results against theoretical signals using MATLAB.

For convenience, here we use the MATLAB Engine API for Python to load the MATLAB code, which can be installed following the instructions at this [link](https://www.mathworks.com/help/matlab/matlab_external/python-setup-script-to-install-matlab-engine-api.html).

In [ ]:
import matlab.engine
eng = matlab.engine.start_matlab()
eng.cd(r'../theoretical_validation', nargout=0)

The function call and parameters are as follows:
[t, back, freq, back_f] = FluidSphere(theta, r_rev, r_src, a, rho1, c1, rho0, c0, source_signal, t_step)
Input parameters:

    theta         - Scattering angle in radians; theta=0 implies backscattering.
    r_rev         - Distance from the receiver to the center of the scatterer (m).
    r_src         - Distance from the planar source to the center of the scatterer (m).
    a             - Radius of the spherical scatterer (m).
    rho1          - Density of the scatterer (kg/m^3).
    c1            - Compressional wave speed of the scatterer (m/s).
    rho0          - Density of the background medium (kg/m^3).
    c0            - Compressional wave speed of the background medium (m/s).
    source_signal - The incident source signal, expected to be a 1xNt array.

Output parameters:

    t             - Time vector corresponding to the source signal.
    back          - Time-domain scattered signal, matching the length of the source signal.
    freq          - Frequency vector for the frequency-domain analysis.
    back_f        - Frequency-domain scattered signal, returned as a 1xNf complex array.

In [ ]:
source_signal = matlab.double(np.expand_dims(src.wavelet, axis=0))
theta = 0. # theta=0 implies backscattering
t, back = eng.FluidSphere(theta, r_rec, r_src, a, rho_scatter, vp_scatter, rho_background, vp_background, source_signal, dt)

In case you don't have MATLAB installed, we also have a presaved data for theoretical scattered signal.

In [ ]:
import scipy
mat = scipy.io.loadmat('theoretical_scatter_signal_acoustic_3d.mat')
t = mat['t']
t = t.squeeze()
theoretical_signal = mat['back']
theoretical_signal = theoretical_signal.squeeze()

Compare the simulated RF with theoretical RF.

In [ ]:
plt.figure(figsize=(12,8))
plt.plot(time, normalized_scatter_signal, label='Simulation')
plt.plot(t, theoretical_signal, label='Theory')
plt.title('Scattered signal')
plt.xlabel('time (s)')
plt.xlim((3e-6, 7e-6))
plt.legend()
plt.show()